In [ ]:
#imports
import os
import shutil
import tensorflow as tf
import numpy as np
from PIL import Image
import cv2
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import array_to_img, img_to_array, load_img
from tqdm import tqdm

2025-04-08 19:45:20.399872: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-08 19:45:20.409664: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-08 19:45:20.419960: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8473] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-08 19:45:20.423302: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1471] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-08 19:45:20.431985: I tensorflow/core/platform/cpu_feature_guar

In [ ]:
dataPath = "../trainingData"

In [ ]:
# import images
#--- words.txt ---------------------------------------------------------------#
#
# iam database word information
#
# format: a01-000u-00-00 ok 154 1 408 768 27 51 AT A
#
#     a01-000u-00-00  -> word id for line 00 in form a01-000u
#     ok              -> result of word segmentation
#                            ok: word was correctly
#                            er: segmentation of word can be bad
#
#     154             -> graylevel to binarize the line containing this word
#     1               -> number of components for this word NOTE: dont think this exists idk why
#     408 768 27 51   -> bounding box around this word in x,y,w,h format
#     AT              -> the grammatical tag for this word, see the
#                        file tagset.txt for an explanation
#     A               -> the transcription for this word
#
class img:
    def __init__ (this,letter,p1,p2):
        this.letter = letter
        this.inPath = p1
        this.savePath = p2
    
images = []

In [ ]:
#import directory for dataset 1 - IAM



In [ ]:
#To add a pixel of border around each picture, because zooming out uses the edge pixels
def addBorder (path):
    t = cv2.imread(dataPath+path+".png")
    pic = np.array(t)
    x,y,z = pic.shape #ALSO change colour here if needed
    
    column = np.array([[[255,255,255]] for i in range (x)]) # to add 1 pixel of white around the picture, because the picture is cropped to exactly where the side contains black and zoom function does not take that well
    row = np.array([[[255,255,255] for i in range (y+2)]])
    pic = np.concatenate([column,pic,column], axis=1)
    pic = np.concatenate([row,pic,row],axis=0)
    cv2.imwrite(dataPath+path+".png",pic)
    return pic


In [ ]:
#import directory for dataset 1 - EMNIST
images = []
directory = open(dataPath + "/ascii/words.txt","r").read().splitlines()
len(directory)
directory = directory[18:] #first 18 lines are instructions
for i in directory:
    data = i.split(" ")
    a = data[0].split("-")
    loc = "/"+a[0]+"/"+a[0]+"-"+a[1]+"/"+data[0]
    letter = data[-1]
    if len(letter) == 1:
        images.append(img(letter,"/words"+loc,"/IAM"+loc))
print(len(images))

if open("log.txt","r").read() == "":
    open("log.txt","w").write("added border")
    for img_ in images:
        try:
            addBorder(img_.inPath)
        except:
            print(dataPath+img_.inPath+".png")



18079


In [ ]:
for i in range (5):
    print (images[i].inPath,letter,images[i].savePath)

/words/a01/a01-000u/a01-000u-00-00 ? /IAM/a01/a01-000u/a01-000u-00-00
/words/a01/a01-000u/a01-000u-02-05 ? /IAM/a01/a01-000u/a01-000u-02-05
/words/a01/a01-000u/a01-000u-03-02 ? /IAM/a01/a01-000u/a01-000u-03-02
/words/a01/a01-000u/a01-000u-04-02 ? /IAM/a01/a01-000u/a01-000u-04-02
/words/a01/a01-000u/a01-000u-06-01 ? /IAM/a01/a01-000u/a01-000u-06-01


In [ ]:
idg = ImageDataGenerator(zoom_range = [1,2],rotation_range = 30,width_shift_range=0.1,height_shift_range=0.1,horizontal_flip = True, vertical_flip = True)

def augment (path,savePath):
    # Converting the input sample image to an array
    # Reshaping the input image
    datagen = idg
    # Loading a sample image 
    img = load_img(path) 
    # Converting the input sample image to an array
    x = img_to_array(img)
    # Reshaping the input image
    x = x.reshape((1, ) + x.shape) 
    
    # Generating and saving 5 augmented samples 
    # using the above defined parameters. 
    i = 0
    if not os.path.exists(savePath):
        os.makedirs(savePath)
    else:
        if len(os.listdir(savePath)) > 0:
            return
    for batch in datagen.flow(x, batch_size = 1,
                            save_to_dir =savePath, 
                            save_prefix ='image', save_format ='jpeg'):
        i += 1
        if i >= 5:
            break

In [ ]:
#execute augment function, will take long time
#change
idg = ImageDataGenerator(zoom_range = [1,2],rotation_range = 30,width_shift_range=0.1,height_shift_range=0.1,horizontal_flip = True, vertical_flip = True,brightness_range=[0.5,1])

last = ""
total = len(images)
last = 0
imagesCopy = images.copy()
count = len(open("progress.txt","r").read())
progress = open("progress.txt","a")
for i in tqdm (range(count,len(images))):
    img_ = images[i]
    count +=1
    augment(dataPath+img_.inPath+".png",dataPath+"/augmentedWords"+img_.savePath)
    progress.write(".")
progress.close()
open("progress.txt","w").close()    

0it [00:00, ?it/s]


In [ ]:
#validation
doesntWork = []
for i in tqdm (range(count,len(images))):
    img_ = images[i]
    try:
        a = os.listdir(dataPath+"/augmentedWords"+img_.savePath)
        if len (a) < 5:
            doesntWork.append(img_)
    except:
        doesntWork.append(img_)
print (doesntWork)
for img_ in doesntWork:
    augment(dataPath+img_.inPath+".png",dataPath+"/augmentedWords"+img_.savePath)


0it [00:00, ?it/s]

[]


In [ ]:
print(len(images))

18079


In [ ]:
images = []
dir = os.listdir(dataPath + "/by_class/")
for i in dir:
    letter = chr(int("0x"+i,0))
    subDir = os.listdir(dataPath + "/by_class/"+i)
    for j in subDir:
        if j[-4:] != ".mit":
            imgDir = os.listdir(dataPath + "/by_class/"+i+"/"+j)
            for k in imgDir:
                images.append(img(letter,"/by_class/"+i+"/"+j+"/"+k[:-4],"/EMNIST/"+i+"/"+j+"/"+k[:-4]))

print (len(images))

1545923


In [ ]:
#execute augment function, will take long time
#change
idg = ImageDataGenerator(zoom_range = [0.4,0.8],rotation_range = 30,width_shift_range=0.1,height_shift_range=0.1,horizontal_flip = True, vertical_flip = True,brightness_range=[0.5,1])

last = ""
total = len(images)
last = 0
imagesCopy = images.copy()
count = len(open("progress.txt","r").read())
progress = open("progress.txt","a")
for i in tqdm (range(count,len(images))):
    img_ = images[i]
    count +=1
    augment(dataPath+img_.inPath+".png",dataPath+"/augmentedWords"+img_.savePath)
    progress.write(".")
progress.close()
open("progress.txt","w").close()      

 14%|█████████████████████▍                                                                                                                                 | 219102/1545923 [25:59<2:42:54, 135.74it/s]

In [ ]:
#validation
doesntWork = []
for i in tqdm (range(count,len(images))):
    img_ = images[i]
    try:
        a = os.listdir(dataPath+"/augmentedWords"+img_.savePath)
        if len (a) < 5:
            doesntWork.append(img_)
    except:
        doesntWork.append(img_)
print (doesntWork)
for img_ in doesntWork:
    augment(dataPath+img_.inPath+".png",dataPath+"/augmentedWords"+img_.savePath)


In [2]:
dataPath = "../trainingData"

In [3]:
# import images
#--- words.txt ---------------------------------------------------------------#
#
# iam database word information
#
# format: a01-000u-00-00 ok 154 1 408 768 27 51 AT A
#
#     a01-000u-00-00  -> word id for line 00 in form a01-000u
#     ok              -> result of word segmentation
#                            ok: word was correctly
#                            er: segmentation of word can be bad
#
#     154             -> graylevel to binarize the line containing this word
#     1               -> number of components for this word NOTE: dont think this exists idk why
#     408 768 27 51   -> bounding box around this word in x,y,w,h format
#     AT              -> the grammatical tag for this word, see the
#                        file tagset.txt for an explanation
#     A               -> the transcription for this word
#
class img:
    def __init__ (this,letter,p1,p2):
        this.letter = letter
        this.inPath = p1
        this.savePath = p2
    
images = []

In [4]:
#import directory for dataset 1 - IAM



In [5]:
#To add a pixel of border around each picture, because zooming out uses the edge pixels
def addBorder (path):
    t = cv2.imread(dataPath+path+".png")
    pic = np.array(t)
    x,y,z = pic.shape #ALSO change colour here if needed
    
    column = np.array([[[255,255,255]] for i in range (x)]) # to add 1 pixel of white around the picture, because the picture is cropped to exactly where the side contains black and zoom function does not take that well
    row = np.array([[[255,255,255] for i in range (y+2)]])
    pic = np.concatenate([column,pic,column], axis=1)
    pic = np.concatenate([row,pic,row],axis=0)
    cv2.imwrite(dataPath+path+".png",pic)
    return pic


In [6]:
#import directory for dataset 1 - EMNIST
images = []
directory = open(dataPath + "/ascii/words.txt","r").read().splitlines()
len(directory)
directory = directory[18:] #first 18 lines are instructions
for i in directory:
    data = i.split(" ")
    a = data[0].split("-")
    loc = "/"+a[0]+"/"+a[0]+"-"+a[1]+"/"+data[0]
    letter = data[-1]
    if len(letter) == 1:
        images.append(img(letter,"/words"+loc,"/IAM"+loc))
print(len(images))

if open("log.txt","r").read() == "":
    open("log.txt","w").write("added border")
    for img_ in images:
        try:
            addBorder(img_.inPath)
        except:
            print(dataPath+img_.inPath+".png")



18079


In [7]:
for i in range (5):
    print (images[i].inPath,letter,images[i].savePath)

/words/a01/a01-000u/a01-000u-00-00 ? /IAM/a01/a01-000u/a01-000u-00-00
/words/a01/a01-000u/a01-000u-02-05 ? /IAM/a01/a01-000u/a01-000u-02-05
/words/a01/a01-000u/a01-000u-03-02 ? /IAM/a01/a01-000u/a01-000u-03-02
/words/a01/a01-000u/a01-000u-04-02 ? /IAM/a01/a01-000u/a01-000u-04-02
/words/a01/a01-000u/a01-000u-06-01 ? /IAM/a01/a01-000u/a01-000u-06-01


In [8]:
idg = ImageDataGenerator(zoom_range = [1,2],rotation_range = 30,width_shift_range=0.1,height_shift_range=0.1,horizontal_flip = True, vertical_flip = True)

def augment (path,savePath):
    # Converting the input sample image to an array
    # Reshaping the input image
    datagen = idg
    # Loading a sample image 
    img = load_img(path) 
    # Converting the input sample image to an array
    x = img_to_array(img)
    # Reshaping the input image
    x = x.reshape((1, ) + x.shape) 
    
    # Generating and saving 5 augmented samples 
    # using the above defined parameters. 
    i = 0
    if not os.path.exists(savePath):
        os.makedirs(savePath)
    else:
        if len(os.listdir(savePath)) > 0:
            return
    for batch in datagen.flow(x, batch_size = 1,
                            save_to_dir =savePath, 
                            save_prefix ='image', save_format ='jpeg'):
        i += 1
        if i >= 5:
            break

In [9]:
#execute augment function, will take long time
#change
idg = ImageDataGenerator(zoom_range = [1,2],rotation_range = 30,width_shift_range=0.1,height_shift_range=0.1,horizontal_flip = True, vertical_flip = True,brightness_range=[0.5,1])

last = ""
total = len(images)
last = 0
imagesCopy = images.copy()
count = len(open("progress.txt","r").read())
progress = open("progress.txt","a")
for i in tqdm (range(count,len(images))):
    img_ = images[i]
    count +=1
    augment(dataPath+img_.inPath+".png",dataPath+"/augmentedWords"+img_.savePath)
    progress.write(".")
progress.close()
open("progress.txt","w").close()    

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18079/18079 [00:34<00:00, 523.00it/s]


In [10]:
#validation
doesntWork = []
for i in tqdm (range(count,len(images))):
    img_ = images[i]
    try:
        a = os.listdir(dataPath+"/augmentedWords"+img_.savePath)
        if len (a) < 5:
            doesntWork.append(img_)
    except:
        doesntWork.append(img_)
print (doesntWork)
for img_ in doesntWork:
    augment(dataPath+img_.inPath+".png",dataPath+"/augmentedWords"+img_.savePath)


0it [00:00, ?it/s]

[]


In [11]:
print(len(images))

18079


In [12]:
images = []
dir = os.listdir(dataPath + "/by_class/")
for i in dir:
    letter = chr(int("0x"+i,0))
    subDir = os.listdir(dataPath + "/by_class/"+i)
    for j in subDir:
        if j[-4:] != ".mit":
            imgDir = os.listdir(dataPath + "/by_class/"+i+"/"+j)
            for k in imgDir:
                images.append(img(letter,"/by_class/"+i+"/"+j+"/"+k[:-4],"/EMNIST/"+i+"/"+j+"/"+k[:-4]))

print (len(images))

1545923


In [13]:
#execute augment function, will take long time
#change
idg = ImageDataGenerator(zoom_range = [0.4,0.8],rotation_range = 30,width_shift_range=0.1,height_shift_range=0.1,horizontal_flip = True, vertical_flip = True,brightness_range=[0.5,1])

last = ""
total = len(images)
last = 0
imagesCopy = images.copy()
count = len(open("progress.txt","r").read())
progress = open("progress.txt","a")
for i in tqdm (range(count,len(images))):
    img_ = images[i]
    count +=1
    augment(dataPath+img_.inPath+".png",dataPath+"/augmentedWords"+img_.savePath)
    progress.write(".")
progress.close()
open("progress.txt","w").close()      

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1545923/1545923 [3:04:25<00:00, 139.70it/s]


In [14]:
#validation
doesntWork = []
for i in tqdm (range(count,len(images))):
    img_ = images[i]
    try:
        a = os.listdir(dataPath+"/augmentedWords"+img_.savePath)
        if len (a) < 5:
            doesntWork.append(img_)
    except:
        doesntWork.append(img_)
print (doesntWork)
for img_ in doesntWork:
    augment(dataPath+img_.inPath+".png",dataPath+"/augmentedWords"+img_.savePath)


0it [00:00, ?it/s]

[]
